In [1]:
# Whisper 사용하여 STT 실행
import os
import whisper
import sounddevice as sd
import numpy as np
import time
#import threading                    # 비동거 작업이나, 동시에 여러 작업 처리
import os
import streamlit as st
import pandas as pd
from dotenv import load_dotenv
import base64
import pyttsx3

# GPT-4로 TTS 실행
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from PIL import Image

In [2]:
# whisper 모델 로드
model = whisper.load_model("base", device="cpu")

In [3]:
# 환경변수 로드
load_dotenv()
API_KEY = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

In [12]:
# Langchain 설정
# Langchain setup
llm = ChatOpenAI(model='gpt-4', temperature=0.7)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # separator로 올바르게 설정
    chunk_size=600,
    chunk_overlap=100,
)


In [14]:
# whisper를 통한 stt
def audio_stt():
    samplerate = 16000
    duration = 3            # 초기설정 3초

    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()

    # whisper를 사용하여 stt
    audio_input = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio_input, n_mel=model.dims.n_mels).to(model.device)
    result = model.transcribe(mel)
    return result['text']


In [16]:
# 음성 감지 시감 설정
def listen_until_silence(silence_threshold=0.02, max_duration=10):
    """
    사용자의 음성을 듣고, 말하는 중이 아니면 녹음을 종료한다.
    silence_threshold : 음성이 감지되지 않는 경우로 판단하는 임계값
    amx_duration : 최대 녹음 시간(s)
    """

    samplerate = 16000
    silence_duration = 0
    audio_data = []

    with sd.InputStream(callback=lambda indata, frames, time, status: audio_data.append(indata), channels=1, samplerate=samplerate, dtype='int16'):
        print("잠시만 기다려 주세요..")
        start_time = time.time()

        while True:
            if time.time() - start_time > max_duration:
                break   # 최대 녹음 시간 초과 시 종료

            # 실시간 오디오 신호의 평균 에너지
            energy = np.linalg.norm(audio_data[-1])**2
            if energy < silence_threshold:
                silence_duration += 1
                if silence_duration > 2:        # 2초 이상 소리가 없으면 멈춤
                    print("응답 생성 중입니다. 잠시만 기다려 주세요..")
                    break

                else:
                    silence_duration = 0        # 음성 감지되면 리셋

    
    # 받은 음성 데이터를 whisper로 변환
    audio = np.concatenate(audio_data, axis=0)
    audio_input = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio_input, n_mels=model.dims.n_mels).to(model.device)
    result = model.transcribe(mel)

    return result['text']


In [17]:
# gpt-4를 사용하여 질문에 대한 답변 생성
def ask_gpt(user_question):
    prompt = ChatPromptTemplate.from_messages(
        [(
            "system", "당신은 차량용 챗봇 Selena 입니다."), ("user", user_question)
        ])

    result = llm.invoke({"question": user_question, "context": retriever})
    return result.content    

In [18]:
# tts
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [19]:
# selena 인식 함수
def detect_keyword(audio):
    audio_input = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrigram(audio_input, n_mels=model.dims.n_mels).to(model.device)
    result = model.transcribe(mel)

    if "Selena" in result['text'].lower():
        return True
    return False

In [20]:
# 음성 처리 및 명령 처리 함수
def process_audio_command():
    # 사용자 음성 인식 후 질문 받기
    print("")
    audio = listen_until_silence()

    # 음성이 selena를 포함하는 지확인
    if detect_keyword(audio):
        print("안녕하세요")

        # 사용자가 질문할 때 까지 대기
        question = listen_until_silence()
        print(f"질문 : {question}")

        # gpt-4로 답변 생성
        answer = ask_gpt(question)
        print(f"답변 : {answer}")

        # 음성으로 답변 전달
        speak(answer)

In [21]:
# streamlit UI
st.set_page_config(layout="wide", page_title="AI Assistant")
st.title("AI Assistant")

2025-02-17 16:19:34.381 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:19:34.384 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:19:34.468 
  command:

    streamlit run c:\Users\INO04\anaconda3\envs\qwer\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-17 16:19:34.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [23]:
# 질문 입력 및 답변
question = st.text_input("편하게 질문하세요", value=st.session_state.get("question", ""))

if question and st.button('Submit'):
    st.session_state["question"] = question
    answer = ask_gpt(question)
    st.write(answer)
    speak(answer)

if __name__ =="__main__":
    #음성처리 및 질문-답변 과정을 동기적으로 실행
    process_audio_command()

2025-02-17 16:20:13.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.440 Session state does not function when running a script without `streamlit run`
2025-02-17 16:20:13.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.442 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.446 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 16:20:13.446 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


PortAudioError: Error querying device -1

In [ ]:
from RealtimeSTT import AudioTextRecorder               # AudioTextRecorder : 실시간 STT 처리
import pyautogui

# 키워드 감지 함수
def detect_keyword(audio):
    # 사용자 음성 STT
    audio = whisper.pad_or_trim(audio)          # audio 데이터 padding or trim
    mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)         # audio 데이터를 멜 스펙트로그램으로 변환
    result = model.transcribe(mel)      # mel_spectrogram을 텍스트로 변환
    #user_input = result['text']

    # 감지된 result에서 Selena 찾기
    if "Selena" in result['text'].lower():
        return True
    return False


# process_command : 특정 키워드를 감지했을 때 호출되며, 사용자의 음성 명령 처리
def process_command(text):
    print(f"질문: {text}")
    response = respond_to_command(text)
    print(response)
    pyautogui.typewrite(response)

# respond_to_command : 질문에 대한 응답 제공
def respond_to_command(command):
    pass

In [1]:
# 필요한 모듈 임포트
import whisper
import os
import pyttsx3
import threading
import base64
import streamlit as st
#from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from RealtimeSTT import AudioToTextRecorder
import pyautogui


c:\Users\INO04\anaconda3\envs\qwer\Lib\site-packages\langchain_community\chat_models\anthropic.py:23: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.llms.anthropic import _AnthropicCommon
c:\Users\INO04\anaconda3\envs\qwer\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.10/u/custom-json-schema

In [2]:
# Whisper 모델 로드
model = whisper.load_model("base", device="cpu")

In [9]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
API_KEY = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

In [ ]:
# Langchain 설정
llm = ChatOpenAI(model="gpt-4", temperature=0.7)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # separator로 올바르게 설정
    chunk_size=600,
    chunk_overlap=100,
)
retriever = FAISS.load_local("./refer.txt", OpenAIEmbeddings())



RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open refer.txt\index.faiss for reading: No such file or directory

In [ ]:


# 사용자 질문을 처리하는 함수
def ask_gpt(user_question):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an intelligent assistant."),
            ("human", "{question}")
        ]
    )

    chain = (
        {
            "context": retriever,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
    )

    result = chain.invoke(user_question)
    return result.content


# 음성을 합성하는 함수 (TTS)
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()


# 이미지 Base64 변환 함수
def get_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode()



In [ ]:

# Streamlit 세션 상태 초기화
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []
if 'question' not in st.session_state:
    st.session_state.question = ""
if 'mode' not in st.session_state:
    st.session_state.mode = "voice"  # 기본 모드 = 음성


# 이미지 경로 설정
ai_avatar = "karina_1-removebg-preview.png"  # AI 아바타 이미지
user_avatar = "사람이미지_1.jpg"  # 사용자 아바타 이미지


# CSS 스타일 정의 (if __name__ == "__main__" 밖에 위치)
st.markdown("""
<style>
.chat-container {
    display: flex;
    align-items: start;
    margin-bottom: 20px;
    width: 100%;
}

.chat-image {
    width: 100px;
    height: 150px;
    border-radius: 10%;
    margin: 0 15px;
    object-fit : cover;
}
.chat-image.ai {
    flex-direction: row-reverse;
    align: right;
    align-items: flex-start;
    justify-content: flex-end;
}
.chat-message {
    background-color: #f0f2f6;
    padding: 10px;
    border-radius: 10px;
    max-width: 80%;
}

.chat-container.ai {
    flex-direction: row-reverse;
    text-align: auto;
}         
            
.chat-message.ai {
    margin-right: 0;
    text-align: auto;
}   

</style>
""", unsafe_allow_html=True)

# 질문 입력 부분
question = st.text_input("언제든 편하게 물어보세요", value=st.session_state.question, key="user_input")

# 질문에 대한 답변을 생성하는 버튼
if question and (st.button('답변') or question != st.session_state.get('previous_question', '')):
    st.session_state['previous_question'] = question
    answer = ask_gpt(question)  # GPT-3 모델을 호출하여 답변을 받습니다.

    st.session_state.chat_history.append(f"Question: {question}")
    st.session_state.chat_history.append(f"Answer: {answer}")
    st.session_state.question = ""  # 입력 필드 클리어

    # 대화 내역 표시
    for message in st.session_state.chat_history:
        if message.startswith("Question:"):
            # 사용자 메시지
            st.markdown(
                f"""
                <div class="chat-container">
                    <img src="data:image/jpeg;base64,{get_image_base64(user_avatar)}" class="chat-image">
                    <div class="chat-message">{message}</div>
                </div>
                """,
                unsafe_allow_html=True
            )
        else: 
            # ai 답변
            st.markdown(
                f"""
                <div class="chat-container ai">
                    <img src="data:image/jpeg;base64,{get_image_base64(ai_avatar)}" class="chat-image">
                    <div class="chat-message ai">{message}</div>
                </div>
                """, 
                unsafe_allow_html=True
            )

    # 자동 스크롤 아래로
    st.markdown("<script>window.scrollTo(0, document.body.scrollHeight);</script>", unsafe_allow_html=True)

else:
    st.error("Please enter a question.")

# Streamlit 페이지 설정
st.set_page_config(layout="wide", page_title="AI Assistant")
st.title("AI Assistant")

# 음성 인식 및 처리 (STT)
def listen_for_audio():
    recorder = AudioToTextRecorder()
    while True:
        recorder.text(process_command)

# STT 명령을 처리하는 함수
def process_command(text):
    print(f"사용자 요구사항: {text}")
    response = respond_to_command(text)
    print(response)
    pyautogui.typewrite(response)

# 명령어에 대한 응답을 처리하는 함수
def respond_to_command(command):
    if "날씨" in command:
        return "오늘의 날씨는 맑습니다."
    elif "시간" in command:
        return "현재 시간은 3시입니다."
    else:
        return "요청을 이해하지 못했습니다."

# 자바스크립트 코드 (복사 및 평가 기능)
st.markdown("""
<div id="scroll-top" onclick="window.scrollTo(0,0)">⬆️</div>
<div id="scroll-bottom" onclick="window.scrollTo(0,document.body.scrollHeight)">⬇️</div>
<script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(() => {
            alert('Copied to clipboard');
        });
    }

    function evaluateResponse(evaluation) {
        alert('You rated this response as: ' + evaluation);
    }
</script>
""", unsafe_allow_html=True)

# `__main__`에서 음성 인식을 위한 스레드를 시작하는 부분
if __name__ == "__main__":
    audio_thread = threading.Thread(target=listen_for_audio)
    audio_thread.start()
